In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def get_page_html(link: str) -> requests.Response:
    r = requests.get(link)
    if r.status_code != 200:
        raise Exception("banned")
    return r


def get_soup(n_page: int) -> BeautifulSoup:
    link = r"https://habr.com/ru/articles/top/daily/page{}/"
    return BeautifulSoup(get_page_html(link.format(n_page)).text, "html.parser")

def get_headings(soup: BeautifulSoup) -> list[list[str, int, int]]:
    headings = []
    for article in soup.find_all("article"):
        score_index = 2
        
        heading = article.find_all("span")[2].text
        attrs = article.find_all("title")
        if len(attrs) == 6:
            score_index += 1
        rez = re.findall(r"[-+]?(?:\d*\.*\d+)", attrs[score_index].text)[1:]
        rez.insert(0, heading)
        headings.append(rez)
    return headings

In [203]:
a = get_headings(get_soup(1))

In [204]:
pd.DataFrame(a)

,0,1,2
0,Почему эффективной сове не выгодно нанимать да...,60,7
1,"Сам написал, сам поиграл: как работали трёхмер...",45,1
2,Как я изобретал клавиатуру,38,0
3,Мануальные объективы на цифровых камерах: хоро...,34,0
4,Пишем трассировщик лучей для ZX Spectrum,33,1
5,Что находится внутри физического Архива Интернета,31,0
6,10 самых распространенных проблем при линтинге...,27,0
7,Испытания 5 отечественных аккумуляторов в корп...,25.5,0.5
8,"НАСА подтверждает, что 2023 год был самым жарк...",24,2
9,Разговор с Максимом Горшениным о мониторах «Ла...,24,3


In [3]:
strange = get_soup(1).find_all("article")

In [197]:
s = strange[7].find_all("title")[3].text

In [201]:
re.findall(r"[-+]?(?:\d*\.*\d+)", s)

['26', '25.5', '0.5']

In [10]:
strange[0].find_all("a")[2]["href"]

'/ru/articles/788984/'

In [3]:
link = "https://habr.com/ru/articles/788984/"

article = requests.get(link)
article.status_code

200

In [4]:
sp = BeautifulSoup(article.text, "html.parser")

In [7]:
sp

<!DOCTYPE html>

<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Пишем трассировщик лучей для ZX Spectrum / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U

In [18]:
contents = sp.find("div", {"class": "article-formatted-body article-formatted-body article-formatted-body_version-2"})

In [57]:
contents.find_all(["p", "code", "h3", "pre"])[0]

<p>Я люблю трассировщики лучей и даже посвятил им <a href="https://gabrielgambetta.com/computer-graphics-from-scratch/02-basic-raytracing.html" rel="noopener noreferrer nofollow">половину своей книги</a>. Менее известна моя любовь к <a href="https://en.wikipedia.org/wiki/ZX_Spectrum" rel="noopener noreferrer nofollow">ZX Spectrum</a> — домашнему компьютеру 1982 года, с которым я вырос и с которого начался мой интерес к графике и программированию. По современным стандартам эта машина столь смехотворно слаба (и даже по стандартам 1980-х), поэтому возникает неизбежный вопрос: в какой степени удастся портировать трассировщик лучей из книги Computer Graphics from Scratch на ZX Spectrum?</p>

In [40]:
link_example = BeautifulSoup('<a href="https://gabrielgambetta.com/computer-graphics-from-scratch/02-basic-raytracing.html" rel="noopener noreferrer nofollow">половину своей книги</a>.')

In [50]:
print(link_example.a.text)

половину своей книги


In [135]:
def parse_a_link(soup: BeautifulSoup) -> BeautifulSoup:
    for i, inner in enumerate(soup.p.contents):
        if inner.name == "a":
            link = inner["href"]
            text = inner.text
            soup.p.contents[i] = f"[{text}]({link})"
    return soup

In [138]:
p_ex = BeautifulSoup('<p>Я люблю трассировщики лучей и даже посвятил им <a href="https://gabrielgambetta.com/computer-graphics-from-scratch/02-basic-raytracing.html" rel="noopener noreferrer nofollow">половину своей книги</a>. Менее известна моя любовь к <a href="https://en.wikipedia.org/wiki/ZX_Spectrum" rel="noopener noreferrer nofollow">ZX Spectrum</a> — домашнему компьютеру 1982 года, с которым я вырос и с которого начался мой интерес к графике и программированию. По современным стандартам эта машина столь смехотворно слаба (и даже по стандартам 1980-х), поэтому возникает неизбежный вопрос: в какой степени удастся портировать трассировщик лучей из книги Computer Graphics from Scratch на ZX Spectrum?</p>')

print(parse_a_link(p_ex).text)

Я люблю трассировщики лучей и даже посвятил им половину своей книги. Менее известна моя любовь к ZX Spectrum — домашнему компьютеру 1982 года, с которым я вырос и с которого начался мой интерес к графике и программированию. По современным стандартам эта машина столь смехотворно слаба (и даже по стандартам 1980-х), поэтому возникает неизбежный вопрос: в какой степени удастся портировать трассировщик лучей из книги Computer Graphics from Scratch на ZX Spectrum?


In [194]:
def get_article_text(article_link: str) -> str:
    response = requests.get(article_link)
    if response.status_code != 200:
        raise Exception("foobar")
    soup = BeautifulSoup(response.text, "html.parser")
    
    return "\n".join(a.text for a in soup.find("div",{"class": "article-formatted-body"}).contents)


In [195]:
link = "https://habr.com/ru/companies/beeline_tech/articles/788948/"

get_article_text(link)

'\r\nКогда я был в Сан-Франциско на AI Engineer Summit, то воспользовался возможностью посетить Архив Интернета — физический архив в калифорнийском городе Ричмонде примерно в двадцати минутах езды от Сан-Франциско.\n\r\nЯ купил билет на «экскурсию по закулисью физического архива» 11 октября и прибыл прямо перед началом; я был рад, что не приехал раньше, потому что физический архив находится (вполне логично) на складе в промышленной части Ричмонда. Похоже, больше ничего интересного в окрестностях нет.\n\r\nЯ попросил водителя Uber подбросить меня до парковки со знаком Internet Archive. Но оглядевшись, я не смог найти публичного входа на склад. Рядом стояло ещё несколько озадаченных фанатов истории Интернета, мы неуклюже поздоровались и начали обсуждать, там ли находимся. Вскоре нас заметила пара людей в конце улицы и помахала нам.\n\n\nФизическое хранилище Архива Интернета в Ричмонде.\n\r\nОказалось, что группа людей уже освоилась внутри основного здания, попивая бесплатную колу, пиво и